# Wave2Vec2
https://huggingface.co/docs/transformers/model_doc/wav2vec2

In [1]:
from transformers import AutoTokenizer, AutoFeatureExtractor, AutoModelForCTC
from datasets import load_dataset, Audio
import datasets
import torch

<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileInfo size changed, may indicate binary incompatibility. Expected 64 from C header, got 88 from PyObject
<frozen importlib._bootstrap>:241: RuntimeWarning: pyarrow._fs.FileSelector size changed, may indicate binary incompatibility. Expected 48 from C header, got 72 from PyObject


In [2]:
import os
hfhome_dir=os.path.join(os.sep, 'DATA10T', 'Cache')
#os.environ['TRANSFORMERS_CACHE'] = hfhome_dir
os.environ['HF_HOME'] = hfhome_dir
#os.environ['HF_HUB_CACHE'] = os.path.join(hfhome_dir, 'hub')
os.environ['HF_DATASETS_CACHE'] = hfhome_dir
#HF_HUB_OFFLINE=1
mycache_dir = hfhome_dir

https://huggingface.co/facebook/mms-1b-all

In [4]:
dataset_name = "mozilla-foundation/common_voice_11_0"

# English
stream_data = load_dataset(dataset_name, "en", split="test", cache_dir=mycache_dir, streaming=True) #
stream_data = stream_data.cast_column("audio", Audio(sampling_rate=16000))
en_sample = next(iter(stream_data))["audio"]["array"]

# French
stream_data = load_dataset(dataset_name, "fr", split="test", cache_dir=mycache_dir, streaming=True) #streaming=True
stream_data = stream_data.cast_column("audio", Audio(sampling_rate=16000))
fr_sample = next(iter(stream_data))["audio"]["array"]

# Chinese
stream_data = load_dataset(dataset_name, "zh-CN", split="test", cache_dir=mycache_dir, streaming=True) #streaming=True
stream_data = stream_data.cast_column("audio", Audio(sampling_rate=16000))
zh_sample = next(iter(stream_data))["audio"]["array"]

/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
Reading metadata...: 16354it [00:00, 18240.44it/s]
/home/lkk/miniconda3/envs/mycondapy310/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_11_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_11_0
You can avoid this message in future by passing the 

In [6]:
en_sample.shape

(100800,)

In [7]:
fr_sample.shape

(61056,)

In [8]:
zh_sample.shape

(38016,)

In [10]:
from transformers import Wav2Vec2ForCTC, AutoProcessor
import torch

model_id = "facebook/mms-1b-all"

processor = AutoProcessor.from_pretrained(model_id, cache_dir=mycache_dir)
model = Wav2Vec2ForCTC.from_pretrained(model_id, cache_dir=mycache_dir)

preprocessor_config.json:   0%|          | 0.00/254 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/397 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.34M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/mms-1b-all were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/mms-1b-all and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream

In [11]:
inputs = processor(en_sample, sampling_rate=16_000, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs).logits

ids = torch.argmax(outputs, dim=-1)[0]
transcription = processor.decode(ids)

In [12]:
transcription

'joe keton disapproved of films and buster also had reservations about the media'

In [13]:
processor.tokenizer.set_target_lang("fra")
model.load_adapter("fra")


adapter.fra.safetensors:   0%|          | 0.00/10.2M [00:00<?, ?B/s]

In [14]:
inputs = processor(fr_sample, sampling_rate=16_000, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs).logits

ids = torch.argmax(outputs, dim=-1)[0]
transcription = processor.decode(ids)
transcription

"ce dernier est volé tout au long de l'histoire romaine"

In [15]:
processor.tokenizer.vocab.keys()

dict_keys(['abi', 'abk', 'abp', 'aca', 'acd', 'ace', 'acf', 'ach', 'acn', 'acr', 'acu', 'ade', 'adh', 'adj', 'adx', 'aeu', 'afr', 'agd', 'agg', 'agn', 'agr', 'agu', 'agx', 'aha', 'ahk', 'aia', 'aka', 'akb', 'ake', 'akp', 'alj', 'alp', 'alt', 'alz', 'ame', 'amf', 'amh', 'ami', 'amk', 'ann', 'any', 'aoz', 'apb', 'apr', 'ara', 'arl', 'asa', 'asg', 'asm', 'ast', 'ata', 'atb', 'atg', 'ati', 'atq', 'ava', 'avn', 'avu', 'awa', 'awb', 'ayo', 'ayr', 'ayz', 'azb', 'azg', 'azj-script_cyrillic', 'azj-script_latin', 'azz', 'bak', 'bam', 'ban', 'bao', 'bas', 'bav', 'bba', 'bbb', 'bbc', 'bbo', 'bcc-script_arabic', 'bcc-script_latin', 'bcl', 'bcw', 'bdg', 'bdh', 'bdq', 'bdu', 'bdv', 'beh', 'bel', 'bem', 'ben', 'bep', 'bex', 'bfa', 'bfo', 'bfy', 'bfz', 'bgc', 'bgq', 'bgr', 'bgt', 'bgw', 'bha', 'bht', 'bhz', 'bib', 'bim', 'bis', 'biv', 'bjr', 'bjv', 'bjw', 'bjz', 'bkd', 'bkv', 'blh', 'blt', 'blx', 'blz', 'bmq', 'bmr', 'bmu', 'bmv', 'bng', 'bno', 'bnp', 'boa', 'bod', 'boj', 'bom', 'bor', 'bos', 'bov', 'b

In [ ]:
#cmn https://dl.fbaipublicfiles.com/mms/misc/language_coverage_mms.html

In [17]:
processor.tokenizer.set_target_lang("cmn-script_simplified")
model.load_adapter("cmn-script_simplified")

(…)dapter.cmn-script_simplified.safetensors:   0%|          | 0.00/31.7M [00:00<?, ?B/s]

In [18]:
inputs = processor(zh_sample, sampling_rate=16_000, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs).logits

ids = torch.argmax(outputs, dim=-1)[0]
transcription = processor.decode(ids)
transcription

'法'